In [1]:
import pandas as pd
import numpy as np
from math import log

In [2]:
data = pd.read_csv('datasets/ml.csv')

In [3]:
data1 = data

In [4]:
print(data)

     outlook  temp humidity    wind play
0      Sunny   Hot     High    Weak   No
1      Sunny   Hot     High  Strong   No
2   Overcast   Hot     High    Weak  Yes
3       Rain  Mild     High    Weak  Yes
4       Rain  Cool   Normal    Weak  Yes
5       Rain  Cool   Normal  Strong   No
6   Overcast  Cool   Normal  Strong  Yes
7      Sunny  Mild     High    Weak   No
8      Sunny  Cool   Normal    Weak  Yes
9       Rain  Mild   Normal    Weak  Yes
10     Sunny  Mild   Normal  Strong  Yes
11  Overcast  Mild     High  Strong  Yes
12  Overcast   Hot   Normal    Weak  Yes
13      Rain  Mild     High  Strong   No


In [5]:
def entropy(*p):
    ret = 0
    for i in p:
        if i != 0:
            ret += (-1)*i*log(i,2)
    return ret

In [6]:
entropy(9/14, 5/14)

0.9402859586706309

In [7]:
tree = {}
print(tree)

{}


In [8]:
def splitInfo(data, key):
    ret = 0
    S = len(data)
    for i in set(data[key]):
        x = len(data[data[key] == i])
        v = x/S
        ret += (-1)*(v)*log(v,2)
    return ret
print(splitInfo(data,'wind'))

0.9852281360342516


In [9]:
def root(data,t):
    x = len(data[data['play'] == 'Yes'])/len(data)
    S = entropy(x, 1-x)
    if S == 0:
        if x == 1:
            t['Yes'] = 'Yes'
        else:
            t['No'] = 'No'
        return
    keys = data.keys()[:-1]
    l = len(data)
    entropys = []
    split = []
    for key in keys:
        e = 0
        for i in set(data[key]):
            x = len(data[(data['play'] == 'Yes') & (data[key] == i)])
            y = len(data[data[key] == i])
            p = x/y
            e += (y/l)*entropy(p, 1-p)
        v = splitInfo(data,key)
        split.append(v)
        entropys.append(e)
    print(entropys)
    gain = [S-i for i in entropys]
    print(gain)
    print(split)
    gainratio = [gain[i]/split[i] for i in range(len(gain))]
    print(gainratio)
    m = max(gainratio)
    print(m)
    a = keys[gainratio.index(m)] 
    t[a] = {}
    for i in set(data[a]):
        t[a][i] = {}
        path = t[a][i]
        asdf = data[data[a]==i]
        newdata = asdf.drop([a],axis=1)
        root(newdata,path)

In [10]:
root(data,tree)

[0.6935361388961918, 0.9110633930116763, 0.7884504573082896, 0.8921589282623617]
[0.24674981977443933, 0.02922256565895487, 0.15183550136234159, 0.04812703040826949]
[1.5774062828523452, 1.5566567074628228, 1.0, 0.9852281360342516]
[0.1564275624211753, 0.018772646222418813, 0.15183550136234159, 0.04884861551152082]
0.1564275624211753
[0.9509775004326937, 0.9509775004326937, 0.0]
[0.01997309402197489, 0.01997309402197489, 0.9709505944546686]
[0.9709505944546686, 0.9709505944546686, 0.9709505944546686]
[0.020570659450692974, 0.020570659450692974, 1.0]
1.0
[0.4, 0.0, 0.9509775004326937]
[0.5709505944546686, 0.9709505944546686, 0.01997309402197489]
[1.5219280948873621, 0.9709505944546686, 0.9709505944546686]
[0.37514952012034747, 1.0, 0.020570659450692974]
1.0


In [11]:
print(tree)

{'outlook': {'Overcast': {'Yes': 'Yes'}, 'Rain': {'wind': {'Weak': {'Yes': 'Yes'}, 'Strong': {'No': 'No'}}}, 'Sunny': {'humidity': {'High': {'No': 'No'}, 'Normal': {'Yes': 'Yes'}}}}}


In [12]:
data1 = pd.read_csv('datasets/ml_test.csv')
for d in data1.iterrows():
    t = tree
    key = [i for i in t.keys()]
    while key[0] != 'No' and key[0] != 'Yes':
        val = d[1][key[0]]
        t = t[key[0]]
        t = t[val]
        key = [i for i in t.keys()]
    print(d[1]['play'],key[0])

No Yes
Yes No
Yes Yes
Yes Yes
Yes Yes
Yes Yes
No No
No Yes
No Yes
Yes Yes
Yes No
Yes No
No Yes
Yes Yes


In [13]:
def confusion():
    confmat = np.array([0,0,0,0])
    confmat = confmat.reshape(2,2)
    for d in data1.iterrows():
        t = tree
        key = [i for i in t.keys()]
        while key[0] != 'No' and key[0] != 'Yes':
            val = d[1][key[0]]
            t = t[key[0]]
            t = t[val]
            key = [i for i in t.keys()]
        predy = key[0]
        y = d[1]['play']
        if y == predy:
            if y == 'Yes':
                confmat[0][0] += 1
            else:
                confmat[1][1] += 1
        else:
            if y == 'Yes':
                confmat[1][0] += 1
            else:
                confmat[0][1] += 1
    return confmat

In [14]:
def accuracy(mat):
    return (mat[0][0] + mat[1][1])/(mat[0][0] + mat[0][1] + mat[1][0] + mat[1][1])
def precision(mat):
    return (mat[0][0])/(mat[0][0] + mat[1][0])
def recall(mat):
    return (mat[0][0])/(mat[0][0] + mat[0][1])
def F1Score(p,r):
    return (2*(p*r))/(p + r)

In [15]:
mat = confusion() 
print(mat)
acc = accuracy(mat)
pre = precision(mat)
rec = recall(mat)
fscore = F1Score(pre, rec)
print(acc, pre, rec, fscore)

[[6 4]
 [3 1]]
0.5 0.6666666666666666 0.6 0.631578947368421
